In [ ]:
import time
import random
from aiomultiprocess import Pool

from veedrive.content.utils import create_cache_subfolders

from create_thumbnail_cache import get_all_supported_files, chunk, generate_thumbnails

# MEDIA_PATH = "/nfs4/bbp.epfl.ch/media/DisplayWall/hpc/library/"
MEDIA_PATH = "/nfs4/bbp.epfl.ch/media/DisplayWall"

CACHE_FOLDER = "/tmp/3"
# CACHE_FOLDER = "/nfs4/bbp.epfl.ch/media/DisplayWall/cache"
files =  get_all_supported_files(MEDIA_PATH)

random.shuffle(files)
no_cpu = 72
print(f"[INFO] No of files to generate: {len(files)}")
number_of_files = len(files)

if number_of_files < no_cpu:
    no_cpu = number_of_files

chunk_size = 1
print(f"[INFO] chunk size  is {chunk_size}")
chunked_list = list(chunk(files, chunk_size))

print(f"[INFO] number of chunks is {len(chunked_list)}")

create_cache_subfolders(CACHE_FOLDER)

print(f"[INFO] starting with {no_cpu} cpu")
pool = Pool(processes=no_cpu)

tt = [(x, MEDIA_PATH, CACHE_FOLDER) for x in chunked_list]

t_start = time.perf_counter()

await pool.starmap(
    generate_thumbnails, [(x, MEDIA_PATH, CACHE_FOLDER) for x in chunked_list]
)

pool.terminate()

print(f"[INFO] Elapsed time is {time.perf_counter() - t_start} seconds")


In [ ]:
t_start=time.perf_counter()

await pool.starmap(generate_thumbnails, [(x, MEDIA_PATH, CACHE_FOLDER) for x in chunked_list])
pool.terminate()
print(f"Elapsed time is {time.perf_counter() - t_start} seconds")  
pool.terminate()
